In [52]:
import numpy as np
import pandas as pd
import random
import math
import time

As mentioned in read me 
Default number of hidden states is 4
Default number is also 5, all the elements in visible sequence is between [0,4]
If you have raw sequence on which HMM has to be applied first get number number of unique hidden states and unique visible states and map states and visible states as follows

In [29]:
class hmm2:
    def __init__(self,sequence,number_states=3,number_output=4,epochs=1000):
        
        self.seq=sequence
        self.states=number_states
        self.output=number_output
        self.T=len(sequence)
        self.maxiter=epochs
        self.iter=0
        self.oldlogprob=-math.inf
        self.epochs=epochs
        
        ## Initialize transition, Emmision, Initial Prob matrix
        
        #self.pie=np.empty(number_states)
        #self.pie.fill(1/number_states)
        self.pie=np.random.dirichlet(np.ones(number_states),size=1)[0] #row statistics
        #self.pie=np.random.uniform(0,1,(number_states)) 
        
        self.transition=np.empty((number_states,number_states))       #row statistics
        for i in range(number_states):
            self.transition[i]=np.random.dirichlet(np.ones(number_states),size=1);
        #self.transition=np.random.uniform(0,1,(number_states,number_states)) 
        #self.transition.fill(1/number_states)
        
        self.emmision=np.empty((number_states,number_output))
        
        for i in range(number_states):
            self.emmision[i]=np.random.dirichlet(np.ones(number_output),size=1) #row statistics
            
        #self.emmision=np.random.uniform(0,1,(number_states,number_output)) 
        #self.emmision.fill(1/number_output)
        
        print("pie")
        print(self.pie)
        print("Transition Matrix")
        print(self.transition)
        print("Emmission Matrix")
        print(self.emmision)
        
        ## Initialize the Alpha, Beta, C matrix
        self.alpha=np.zeros(shape=(number_states,self.T))
        self.beta=np.zeros(shape=(number_states,self.T))
        self.C=np.zeros(self.T)
        
        ##Intitialize the gamma and dual gamma matrix
        
        self.dual_gamma=np.zeros(shape=(number_states,number_states,self.T))
        self.gamma=np.zeros(shape=(number_states,self.T))
        
        
    def alpha_pass(self):
        S=self.states
        T=self.T
        
        
        for time in range(T):
            c=0
            for states in range(S):
                self.alpha[states][time]=0
                if(time==0):
                    self.alpha[states][time]=self.pie[states]*self.emmision[states][self.seq[time]]
                else:
                    for pre_state in range(S):
                        self.alpha[states][time]+=self.transition[pre_state][states]*self.alpha[pre_state][time-1]*self.emmision[states][self.seq[time]]
            for states in range(S):
                c+=self.alpha[states][time]
                        
            for states in range(S):
                self.alpha[states][time]=self.alpha[states][time]/c
                
            self.C[time]=1/c  
                
        #print("Alpha")
        #print(self.alpha)
        #print("C")
        #print(self.C)
        
    def beta_pass(self):
        
        S=self.states
        T=self.T
        
        for time in range(T-1,-1,-1):
            c=0
            for states in range(S):
                self.beta[states][time]=0
                if(time==T-1):
                    self.beta[states][time]=self.C[time]  
                    continue
                for nxt_state in range(S):
                       self.beta[states][time]+=self.beta[nxt_state][time+1]*self.transition[states][nxt_state]*self.emmision[nxt_state][self.seq[time+1]]*self.C[time]
        
        #print("BETA")        
        #print(self.beta)
        
    def gamma_pass(self):
        S=self.states
        T=self.T
        
        for t in range(T-1):
            c=0
            c1=0
            for state in range(S):
                self.gamma[state][t]=0
                c1=0
                for state2 in range(S):
                    self.dual_gamma[state][state2][t]=self.alpha[state][t]*self.transition[state][state2]*self.emmision[state2][self.seq[t+1]]*self.beta[state2][t+1]
                    self.gamma[state][t]+=self.dual_gamma[state][state2][t]
                
        for state in range(S):
            self.gamma[state][T-1]=self.alpha[state][T-1]
            
        #print(self.dual_gamma)
        #print(" ")
        #print(" ")
        #print(self.gamma)
        
        
    
        
    def forward_backward(self):
        S=self.states
        T=self.T
        out=self.output
        deno=0
        numo=0
        
        for state in range(S):
            self.pie[state]=self.gamma[state][0]
        
        
        for state_i in range(S):
            deno=0
            for time in range(T-1):
                deno+=self.gamma[state_i][time]
                
            for state_j in range(S):
                numo=0
                for time in range(T-1):
                    numo+=self.dual_gamma[state_i][state_j][time]
                self.transition[state_i][state_j]=numo/deno
                
                
        for state in range(S):
            for v in range(out):
                numo=0
                deno=0
                for time in range(T):
                    if(self.seq[time]==v):
                        numo+=self.gamma[state][time]
                    deno+=self.gamma[state][time]
                self.emmision[state][v]=numo/deno
        #print("Transition_Matrix")
        #print(self.transition)
        #print("Emission_Matrix")
        #print(self.emmision)
        
    def parameter(self):
        return self.gamma
    
    def give_prob(self):
        S=self.states
        T=self.T
        logprob=0
        for time in range(T):
            logprob+=math.log10(self.C[time])
        logprob=-1*logprob
        return logprob
    
    def train(self):
        for epoch in range(self.epochs):
               
            
            self.alpha_pass()
            self.beta_pass()
            self.gamma_pass()
            self.forward_backward()
            newlogprob=self.give_prob()
            self.oldlogprob=newlogprob
            
        print("Training Done, Model Converged")
        print("Transition Matrix")
        print(self.transition)
        print("")
        print("Emmision Matrix")
        print(self.emmision)
        print("")
        print("Initial Matrix")
        print(self.pie)
        
    
    def decoding(self):
        
        return np.argmax(self.gamma, axis=0)
                
    def display(self):
        print("Transition")
        print(self.transition)
        
        print("emmission")
        print(self.emmision)
        
        print("Alpha")
        print(self.alpha)
        print("Beta")
        print(self.beta)
        
        print("gamma")
        print(self.gamma)
        #return self.transition,self.emmision,self.alpha,self.alpha_pass
        
        

In [32]:
se=[]
for x in range(20000):
       se.append(random.randint(0,3))

In [33]:

se=np.array(se)
se

array([0, 0, 3, ..., 2, 1, 0])

In [34]:
k=hmm2(se)

pie
[0.19908917 0.58721572 0.21369512]
Transition Matrix
[[0.2415714  0.72516327 0.03326533]
 [0.58102602 0.3900208  0.02895317]
 [0.49480381 0.11460373 0.39059246]]
Emmission Matrix
[[0.1223515  0.33378858 0.17225359 0.37160632]
 [0.31500505 0.19795761 0.15352508 0.33351226]
 [0.22435966 0.02917742 0.72460245 0.02186047]]


In [35]:
start_time = time.time()
k.alpha_pass()
k.beta_pass()
k.gamma_pass()
k.forward_backward()
print(time.time()-start_time)

1.21075439453125


In [28]:
k.display()

Transition
[[0.06854573 0.50805282 0.42340145]
 [0.14357232 0.04057906 0.81584863]
 [0.08034499 0.58036634 0.33928867]]
emmission
[[0.05913836 0.77466973 0.16619191]
 [0.67477296 0.10344797 0.22177908]
 [0.14422723 0.40014753 0.45562523]]
Alpha
[[0.19865169 0.04817503 0.01080596 ... 0.00634124 0.25283433 0.04561627]
 [0.07296253 0.27606027 0.85852577 ... 0.91743547 0.00666913 0.31943474]
 [0.72838578 0.6757647  0.13066827 ... 0.0762233  0.74049653 0.63494898]]
Beta
[[ 4.35384965  7.98696687  0.63845288 ...  0.66977773  2.46814705
   6.92049636]
 [12.67060856  1.00573338  1.96447282 ...  1.56994508  5.40949022
   6.92049636]
 [ 3.65035706  8.44771161  0.57104688 ...  0.63241194  2.21089719
   6.92049636]]
gamma
[[0.19443603 0.06039358 0.00390205 ... 0.00284518 0.27164075 0.04561627]
 [0.20783009 0.04357864 0.95389497 ... 0.96486293 0.01570411 0.31943474]
 [0.59773388 0.89602778 0.04220298 ... 0.03229189 0.71265514 0.63494898]]


In [10]:
import numpy as np, numpy.random

In [11]:
k.alpha_pass()
k.beta_pass()
k.gamma_pass()
k.forward_backward()

In [13]:
k.decoding()

array([0, 1, 2, ..., 2, 1, 1], dtype=int64)

In [14]:
k.display()

Transition
[[0.04227409 0.90663375 0.05109216]
 [0.15198899 0.13934143 0.70866959]
 [0.23194623 0.41102292 0.35703085]]
emmission
[[0.5705067  0.28360873 0.14588457]
 [0.04830903 0.4307422  0.52094876]
 [0.48414627 0.2499291  0.26592463]]
Alpha
[[0.6836219  0.02867197 0.07647238 ... 0.07418883 0.16439468 0.14383019]
 [0.05971147 0.87743931 0.33412943 ... 0.40487427 0.46498028 0.4941603 ]
 [0.25666663 0.09388872 0.58939819 ... 0.5209369  0.37062504 0.36200951]]
Beta
[[4.00450115 2.95402883 6.03365963 ... 3.84038201 4.07323695 3.10998836]
 [2.41218238 2.18371278 2.57699454 ... 3.04333749 2.73236497 3.10998836]
 [2.91231158 2.43873106 3.63855385 ... 3.46112689 3.26629848 3.10998836]]
gamma
[[0.75433856 0.03798547 0.13308518 ... 0.08581443 0.21253099 0.14383019]
 [0.03968897 0.85932584 0.24835476 ... 0.37112282 0.40324416 0.4941603 ]
 [0.20597247 0.10268868 0.61856005 ... 0.54306275 0.38422486 0.36200951]]


In [15]:
k.alpha_pass()
k.beta_pass()
k.gamma_pass()
k.forward_backward()

In [16]:
k.decoding()

array([0, 1, 2, ..., 2, 1, 1], dtype=int64)

In [17]:
k.alpha_pass()
k.beta_pass()
k.gamma_pass()
k.forward_backward()

In [19]:
k.display()

Transition
[[0.04605486 0.89593345 0.05801169]
 [0.13997754 0.15063681 0.70938565]
 [0.23316887 0.39112968 0.37570146]]
emmission
[[0.54265295 0.29250092 0.16484613]
 [0.06242635 0.43116282 0.50641082]
 [0.47709996 0.24887887 0.27402117]]
Alpha
[[0.81370022 0.02545177 0.07823429 ... 0.0776464  0.1654452  0.14493723]
 [0.06137854 0.89467826 0.31840326 ... 0.41606363 0.4527705  0.49133333]
 [0.12492124 0.07986997 0.60336245 ... 0.50628997 0.3817843  0.36372944]]
Beta
[[3.6547937  2.95384595 5.54206283 ... 3.70725982 4.05850716 3.10126586]
 [2.39353534 2.16182241 2.70976703 ... 2.91231112 2.75387788 3.10126586]
 [2.73746877 2.41310192 3.40875695 ... 3.32052831 3.23993968 3.10126586]]
gamma
[[0.85881897 0.03414119 0.12930725 ... 0.09050029 0.21280448 0.14493723]
 [0.04242587 0.87833386 0.25731419 ... 0.38095453 0.39516918 0.49133333]
 [0.09875516 0.08752496 0.61337856 ... 0.52854519 0.39202634 0.36372944]]


In [36]:
start_time=time.time()
k.train()
print("--- %s seconds elapsed ---" % (time.time()-start_time))

Training Done, Model Converged
Transition Matrix
[[0.26731943 0.65097745 0.08170312]
 [0.49760787 0.4265668  0.07582533]
 [0.70832314 0.06641352 0.22526335]]

Emmision Matrix
[[0.20459267 0.24422635 0.18085584 0.37032514]
 [0.32875187 0.23823173 0.24265603 0.19036037]
 [0.00414878 0.34251863 0.64205355 0.01127904]]

Initial Matrix
[5.78898315e-130 1.00000000e+000 0.00000000e+000]
--- 1219.3511273860931 seconds elapsed ---


In [47]:
m=k.decoding()

In [48]:
s1=0
s2=0
s3=0
for i in m:
    if(i==0):
        s1+=1
        print("State 1")
    elif(i==1):
        s2+=1
        print("State 2")
    else:
        s3+=1
        print("State 3")

State 2
State 2
State 1
State 1
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 1
State 1
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2


State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 1
State 1


State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 1
State 2
State 2


State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 1
State 2


State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 1
State 2
State 2
State 2
State 1
State 2
State 1
State 2
State 1
State 1
State 2
State 2
State 1
State 2
State 2
State 2
State 1
State 1
State 2
State 2
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 1
State 2
State 2
State 2
State 2
State 2
State 1
State 1
State 2
State 1
State 2
State 2
State 2
State 1
State 2
State 2
State 1


In [49]:
s1

6541

In [50]:
s2

13452

In [51]:
s3

7